In [33]:

import numpy as np
import cv2
import json
import Detector
import Distance

In [50]:

# An input file to get stored embeddings
input_file = 'embeddings/embeddings.json'

# model_file = 'models/facenet_weights.h5'

WIDTH = 640
HEIGHT = 480

# Set the cosine similarity threshold for recognition
threshold = Distance.findThreshold(model_name="Facenet", distance_metric="cosine")

# threshold

# threshold = 0.000000001

threshold

# # Load the FaceNet model
# face_net_model = Facenet.loadModel()


0.4

In [51]:
stored_embeddings = {}

# Load the embeddings from the JSON file
with open(input_file, 'r') as json_file:
    stored_data = json.load(json_file)

    # Iterate through the JSON data and convert it to the desired format
    for user_data in stored_data:
        user_name = user_data["name"]
        user_embeddings = user_data["embeddings"]

        # Convert user's embeddings to a flat list
        embeddings_list = []
        for key, embeddings in user_embeddings.items():
            for embedding in embeddings:
                embeddings_list.append(embedding)

        # Store the converted data in the dictionary
        stored_embeddings[user_name] = {
            "embeddings": embeddings_list
        }

print(stored_embeddings)

{'Mussa': {'embeddings': [[-0.1586964875459671, -0.428977906703949, -0.08908072859048843, -0.4541493058204651, 0.37028780579566956, 0.11209093779325485, -0.03748645633459091, -0.19790872931480408, -0.03333413228392601, 0.1930813193321228, 0.23022815585136414, -0.3007429838180542, 0.14622822403907776, -0.3123322129249573, -0.09479223191738129, 0.4439272880554199, 0.3271879553794861, -0.3340646028518677, 0.2641061842441559, -0.530560314655304, 0.18292167782783508, 0.15602950751781464, -0.16998407244682312, -0.021060839295387268, 0.23368801176548004, -0.09452839940786362, 0.48969969153404236, 0.4445571005344391, -0.12250789999961853, -0.02038811519742012, -0.22742381691932678, 0.39198291301727295, 0.021802514791488647, 0.040881939232349396, 0.5121214985847473, -0.4489957094192505, 0.17470943927764893, 0.11615137755870819, 0.48904144763946533, 0.10670693963766098, 0.23641908168792725, -0.003428814932703972, -0.09972687065601349, -0.17340119183063507, -0.439679354429245, -0.5405646562576294

In [52]:
def process_detected_faces(detected_faces, frame):
    # print("detected_faces:", detected_faces)

    if len(detected_faces) > 0:

        for face_obj in detected_faces:

            if face_obj["confidence"] >= 2.0:

                x = face_obj["facial_area"]["x"]
                y = face_obj["facial_area"]["y"]
                w = face_obj["facial_area"]["w"]
                h = face_obj["facial_area"]["h"]
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                # # Extract the face ROI
                # face_roi = frame[y:y+h, x:x+w]
                # # Preprocess the face image for FaceNet
                # face_img = cv2.resize(face_roi, (160, 160))
                # face_img = cv2.resize(face_obj["face"], (160, 160))
                # face_img = np.expand_dims(face_img, axis=0)
                # face_img = (face_img - 127.5) / 128.0  # Normalize

                # Extract embeddings from the face using FaceNet
                # face_embeddings = face_net_model.predict(face_img)
                face_embeddings = []
                _face_embeddings = Detector.represent(
                    img_path=face_obj["face"],
                    enforce_detection=False
                )
                if len(_face_embeddings) > 0:
                    face_embeddings.append(_face_embeddings[0]["embedding"])

                    # Compare the embeddings with stored embeddings for each user using cosine similarity
                    found_similarity = -1.0
                    recognized_face = None

                    for name, stored_user_data in stored_embeddings.items():
                        for stored_embedding in stored_user_data["embeddings"]:

                            source_representation = stored_embedding
                            test_representation = face_embeddings[0]

                            print("source_representation:  ", source_representation)
                            print("test_representation:    ", test_representation)

                            print(f"source_representation: {len(source_representation)}")
                            print(f"test_representation:   {len(test_representation)}")

                            # Check if there is a dimension mismatch
                            if len(test_representation) != len(source_representation):
                                # Calculate the number of zero values to append
                                num_zeros_to_append = len(
                                    source_representation) - len(test_representation)
                                # Append zero values to test_representation
                                test_representation = np.append(
                                    test_representation, np.zeros(num_zeros_to_append))

                            # Calculate cosine distance
                            cosine_distance = Distance.findCosineDistance(source_representation, test_representation)

                            # print(f"Cosine Distance: {cosine_distance:.2f}")
                            print(f"Cosine Distance: {cosine_distance}, threshold: {threshold}")

                            if cosine_distance >= threshold: 
                                found_similarity = cosine_distance
                                recognized_face = name
                                break

                            else:
                                found_similarity = -1.0

                    print(f"similarity: {found_similarity}, threshold: {threshold}")
                    print("--------------------------------------")

                    if recognized_face is not None:  # Check against the threshold for recognition
                        cv2.putText(frame, f"{recognized_face} ({found_similarity})", (x, y-10),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                    else:
                        cv2.putText(frame, 'Unknown', (x, y-10),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)



In [53]:
def recognize_image(img_path):

    image = cv2.imread(img_path)

    # Define the maximum width and height for display
    max_width = 800
    max_height = 600

    # Get the current dimensions of the image
    height, width, _ = image.shape

    # Check if resizing is necessary
    if width > max_width or height > max_height:
        # Calculate the scaling factor to fit within the maximum dimensions
        scale = min(max_width / width, max_height / height)

        # Calculate the new dimensions
        new_width = int(width * scale)
        new_height = int(height * scale)

        # Resize the image
        image = cv2.resize(image, (new_width, new_height))

    detected_faces = Detector.extract_faces(
        img_path=image,
        enforce_detection=False
    )
    for i, face_obj in enumerate(detected_faces):
        print(f'face: {i + 1}, confidence: {face_obj["confidence"]}')

    process_detected_faces(detected_faces=detected_faces, frame=image)

    cv2.imshow('Face Recognition', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    print("-------------------------------------")

recognize_image("input_images\\Mussa\\2022-12-12 (1).jpg")
recognize_image("D:\\Downloads\\ali_gallo_instagram_2.jpg")
recognize_image("D:\\Downloads\\pearl-andersson-soundcloud.jpg")
recognize_image("D:\\Projects\\ml-enthusiast\\face\\input_images\\Mussa\\20200130_172549.jpg")
recognize_image("D:\\Projects\\ml-enthusiast\\face\\input_images\\Mussa\\2022-12-12 (16).jpg")
recognize_image("D:\\Downloads\\Ai Khodijah - singer Islam.png")
recognize_image("D:\\Downloads\\portrait-young-patient-lying-bed-during-clinical-examination-recovering-after-medical-surgery-hospital-ward-sick-woman-looking-into-camera-waiting-healthcare-treatment.jpg")
recognize_image("D:\\Projects\\ml-enthusiast\\face\\input_images\\Mussa\\2022-12-12 (17).jpg")
recognize_image("D:\\Downloads\\FB_IMG_1581885670173.jpg")
recognize_image("D:\\Projects\\ml-enthusiast\\face\\input_images\Mussa\\2022-12-12 (6).jpg")
recognize_image("D:\\Downloads\\FvCGMCrakAMFIMB.png")
recognize_image("D:\\Downloads\\130863182_382785296508190_2107725103963850677_n.jpg")
recognize_image("D:\\Downloads\\IMG_E0196.JPG")
recognize_image("D:\\Downloads\\20230820_121603.jpg")
recognize_image("D:\\Downloads\\07b7ab7c91ac4820b7a535b0ecd9b6c3.jpg")
recognize_image("D:\\Downloads\\Screenshot_20230523_163206_WhatsAppBusiness.jpg")
recognize_image("D:\\Downloads\\2022-12-12 (18).jpg")



face: 1, confidence: 4.661835087055806
1/1 [==============================] - 0s 130ms/step
source_representation:   [-0.1586964875459671, -0.428977906703949, -0.08908072859048843, -0.4541493058204651, 0.37028780579566956, 0.11209093779325485, -0.03748645633459091, -0.19790872931480408, -0.03333413228392601, 0.1930813193321228, 0.23022815585136414, -0.3007429838180542, 0.14622822403907776, -0.3123322129249573, -0.09479223191738129, 0.4439272880554199, 0.3271879553794861, -0.3340646028518677, 0.2641061842441559, -0.530560314655304, 0.18292167782783508, 0.15602950751781464, -0.16998407244682312, -0.021060839295387268, 0.23368801176548004, -0.09452839940786362, 0.48969969153404236, 0.4445571005344391, -0.12250789999961853, -0.02038811519742012, -0.22742381691932678, 0.39198291301727295, 0.021802514791488647, 0.040881939232349396, 0.5121214985847473, -0.4489957094192505, 0.17470943927764893, 0.11615137755870819, 0.48904144763946533, 0.10670693963766098, 0.23641908168792725, -0.003428814932

In [ ]:
def main():
    # Create a VideoCapture object to capture video from the default camera (usually 0)
    cap = cv2.VideoCapture(0)
    cap.set(3,WIDTH)
    cap.set(4,HEIGHT)

    while True:
        # Read a frame from the camera
        _, frame = cap.read()
        
        detected_faces = Detector.extract_faces(
            img_path=image,
            enforce_detection=False
        )
        for i, face_obj in enumerate(detected_faces):
            print(f'face: {i + 1}, confidence: {face_obj["confidence"]}')

        process_detected_faces(detected_faces=detected_faces, frame=frame)
        
        cv2.imshow('Face Recognition', frame)

        # Listen for a key press and check if it's the "Esc" key (27 is the ASCII code for Esc)
        key = cv2.waitKey(1)
        if key == 27:
            break

    # Release the VideoCapture and close all OpenCV windows
    cap.release()
    cv2.destroyAllWindows()

main()


In [39]:
# Replace these with actual face embeddings (NumPy arrays)
source_representation = np.array([-0.16008254885673523, -0.42965245246887207, -0.08791586756706238, -0.45383596420288086, 0.36987555027008057, 0.11270783096551895, -0.038006946444511414, -0.1974789798259735, -0.03306589648127556, 0.1935141235589981, 0.2302180528640747, -0.30009764432907104, 0.14654581248760223, -0.3119507431983948, -0.09544502198696136, 0.4436224699020386, 0.3256615698337555, -0.33390557765960693, 0.26346051692962646, -0.5316306948661804, 0.1834142506122589, 0.1549956351518631, -0.1686103641986847, -0.020361803472042084, 0.2347182035446167, -0.09382089227437973, 0.4907272756099701, 0.4458763897418976, -0.12322874367237091, -0.02241300791501999, -0.23014825582504272, 0.3929699659347534, 0.02003161981701851, 0.04171992093324661, 0.5112512111663818, -0.4482964277267456, 0.1744278073310852, 0.11592582613229752, 0.488915354013443, 0.10592157393693924, 0.2362450361251831, -0.004481358453631401, -0.09867370128631592, -0.17389631271362305, -0.43861931562423706, -0.5397346615791321, 0.02525930106639862, -0.2082960456609726, 0.5320633053779602, 0.269456148147583, -0.49773019552230835, 0.3023086488246918, 0.21460747718811035, -0.4294174015522003, 0.11844071745872498, 0.1253633052110672, 0.5314725637435913, 0.054270755499601364, -0.3316863179206848, -0.906057596206665, -0.44210195541381836, -0.35678285360336304, -0.06184165179729462, 0.7101737856864929, -0.32034537196159363, 0.4858940839767456, 0.3026501536369324, 0.05908270180225372, 0.2842181622982025, -0.4691965878009796, -0.0875568613409996, -0.4234546720981598, 0.19403678178787231, -0.4220426082611084, -0.04387064650654793, -0.10095703601837158, 0.02104450762271881, 0.33574944734573364, -0.2271248996257782, -0.297338604927063, -0.5944415330886841, -0.1837373971939087, 0.17139282822608948, 0.2690936028957367, 0.4185224175453186, 0.3320041298866272, 0.4281994700431824, -0.19138072431087494, -0.6225048303604126, 0.8666657209396362, 0.4038495719432831, 0.0512024387717247, -0.0250396691262722, 0.0399477444589138, 0.7860700488090515, 0.20360465347766876, -0.03611861169338226, -0.2690652012825012, -0.7563657164573669, -0.10011422634124756, -0.2220708429813385, 0.5768916606903076, 0.1844230443239212, -0.33842864632606506, 0.004947535693645477, 0.14233069121837616, -0.1633542776107788, 0.06423656642436981, -0.3463911712169647, -0.4156837463378906, 0.22279468178749084, 0.19906046986579895, 0.3007173538208008, 0.43595367670059204, 0.01978151500225067, 0.11893285810947418, 0.16957294940948486, 0.34449028968811035, 0.197540283203125, -0.0534418448805809, -0.16101142764091492, 0.07004986703395844, 0.30039477348327637, 0.04176343232393265, 0.2815759778022766, -0.5096381902694702, 0.18628551065921783, 0.1794898808002472])
test_representation = np.array([-0.15968625247478485, -0.42924964427948, -0.08815459907054901, -0.45323604345321655, 0.37057429552078247, 0.11197736114263535, -0.03724232316017151, -0.19771645963191986, -0.03301047161221504, 0.19324015080928802, 0.23096191883087158, -0.300538033246994, 0.14682906866073608, -0.3121193051338196, -0.09521158039569855, 0.4436360001564026, 0.32579582929611206, -0.33402353525161743, 0.26275521516799927, -0.5309196710586548, 0.1826157569885254, 0.15595956146717072, -0.16804082691669464, -0.020359262824058533, 0.23453694581985474, -0.09365112334489822, 0.49023863673210144, 0.445359468460083, -0.12376295030117035, -0.02219552733004093, -0.2296285629272461, 0.39204972982406616, 0.020114902406930923, 0.041868485510349274, 0.5122045278549194, -0.4476974904537201, 0.17461444437503815, 0.11559237539768219, 0.488481730222702, 0.1063445582985878, 0.23611482977867126, -0.0036909026093780994, -0.0987219512462616, -0.17368322610855103, -0.4385102689266205, -0.5400220155715942, 0.024670753628015518, -0.20768553018569946, 0.5318990349769592, 0.2707829475402832, -0.49670660495758057, 0.3019949197769165, 0.21484556794166565, -0.43032529950141907, 0.11808404326438904, 0.12524738907814026, 0.5322068333625793, 0.054583776742219925, -0.33160707354545593, -0.9072461128234863, -0.44157832860946655, -0.35711586475372314, -0.06180049479007721, 0.7103531956672668, -0.32100731134414673, 0.4854019284248352, 0.3021053075790405, 0.058439720422029495, 0.2844870984554291, -0.4697500765323639, -0.087823785841465, -0.42427584528923035, 0.1939660608768463, -0.42303580045700073, -0.0441446490585804, -0.10112346708774567, 0.02136075496673584, 0.33584916591644287, -0.22770515084266663, -0.2981024980545044, -0.5946446657180786, -0.18299151957035065, 0.1711488664150238, 0.2694268226623535, 0.4187791645526886, 0.3311644494533539, 0.42875006794929504, -0.19060227274894714, -0.6221834421157837, 0.8669156432151794, 0.40253981947898865, 0.05159049481153488, -0.02490091137588024, 0.04118133708834648, 0.7859834432601929, 0.20264974236488342, -0.03537474572658539, -0.2697853147983551, -0.7567518353462219, -0.09987839311361313, -0.2214249074459076, 0.576863169670105, 0.18424329161643982, -0.338388592004776, 0.005349390208721161, 0.1426307111978531, -0.16239257156848907, 0.06484001874923706, -0.34641337394714355, -0.41620635986328125, 0.22367742657661438, 0.19972752034664154, 0.30129700899124146, 0.4369998872280121, 0.020031994208693504, 0.11921486258506775, 0.1703014224767685, 0.34339481592178345, 0.19754073023796082, -0.05331708490848541, -0.16087742149829865, 0.06967407464981079, 0.299550861120224, 0.040638700127601624, 0.281508207321167, -0.5102768540382385, 0.18530631065368652, 0.17894448339939117])

# Check if there is a dimension mismatch
if len(test_representation) != len(source_representation):
    # Calculate the number of zero values to append
    num_zeros_to_append = len(source_representation) - len(test_representation)
    
    # Append zero values to test_representation
    test_representation = np.append(test_representation, np.zeros(num_zeros_to_append))

# # Check if there is a dimension mismatch
# if len(source_representation) != len(test_representation):
#     # Calculate the number of zero values to append to source_representation
#     num_zeros_to_append = len(test_representation) - len(source_representation)
    
#     # Append zero values to source_representation
#     source_representation = np.append(source_representation, np.zeros(num_zeros_to_append))

# # Determine the target dimensionality (maximum of source and 128)
# target_dimension = max(len(source_representation), 128)

# # Pad both source and test representations to the target dimensionality
# source_representation = np.pad(source_representation, (0, target_dimension - len(source_representation)), mode='constant')
# test_representation = np.pad(test_representation, (0, target_dimension - len(test_representation)), mode='constant')

# Calculate cosine distance
cosine_distance = Distance.findCosineDistance(source_representation, test_representation)

print(f"Cosine Distance: {cosine_distance}")
print(f"Cosine Distance: {cosine_distance:.2f}")



Cosine Distance: 1.4914007959720976e-06
Cosine Distance: 0.00


In [ ]:
# Replace these with actual face embeddings (NumPy arrays)
source_representation = np.array([0.1, 0.2, 0.3])
test_representation = np.array([0.11, 0.18, 0.28])

# Calculate cosine distance
cosine_distance = findCosineDistance(source_representation, test_representation)

print(f"Cosine Distance: {cosine_distance:.2f}")